In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# --- [사용자가 반드시 수정해야 할 설정 부분] ---

# 1. 크롤링할 웹사이트의 URL을 입력하세요.
TARGET_URL = "https://www.greating.co.kr/market/marketDetail?itemId=166994&toggle=meals" 

# 2. 크롬 드라이버 경로를 설정하세요. 
#    (최근 Selenium 버전은 자동으로 드라이버를 찾아주기도 하지만, 문제가 생길 경우 경로 지정)
# DRIVER_PATH = 'C:/path/to/chromedriver.exe' 
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)
# 자동 다운로드를 사용할 경우 아래와 같이 설정 (주석 해제 필요 없음)

# 3. 데이터가 담긴 요소의 CSS 선택자(Selector)를 입력하세요.
#    (예: 게시판의 각 항목, 상품 목록 등)
DATA_ITEM_SELECTOR = "여기에 데이터 목록을 감싸는 요소의 CSS 선택자를 입력하세요 (예: .product-list > .item)"

# 4. 다음 페이지로 넘어가는 버튼의 CSS 선택자를 입력하세요.
#    (예: 페이지 하단의 '다음', 'Next', 또는 '>>' 버튼)
NEXT_PAGE_BUTTON_SELECTOR = "여기에 '다음 페이지' 버튼의 CSS 선택자를 입력하세요 (예: a.next_btn)"

# 5. 최대 크롤링할 페이지 수를 설정하세요. (무한 루프 방지)
MAX_PAGES = 10 

# --- [드라이버 설정 및 초기화] ---

def setup_driver():
    """크롬 드라이버를 설정하고 초기화합니다."""
    try:
        options = webdriver.ChromeOptions()
        # 브라우저 창을 띄우지 않고 백그라운드에서 실행하려면 아래 주석을 해제하세요.
        # options.add_argument('headless') 
        options.add_argument('window-size=1920x1080')
        options.add_argument('disable-gpu')
        options.add_argument('lang=ko_KR')
        driver = webdriver.Chrome(options=options)
        return driver
    except Exception as e:
        print(f"드라이버 설정 중 오류 발생: {e}")
        print("Chrome 브라우저와 Selenium WebDriver 버전을 확인하거나, DRIVER_PATH를 명시적으로 설정해주세요.")
        return None

# --- [스크롤 및 페이지네이션 처리 함수] ---

def scroll_to_bottom(driver, scroll_delay=2):
    """페이지 끝까지 스크롤하여 동적 로딩을 유도합니다 (무한 스크롤 처리)."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    print("페이지 하단으로 스크롤 중...")
    while True:
        # 페이지 하단으로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # 페이지 로딩을 기다림
        time.sleep(scroll_delay)
        
        # 새로운 스크롤 높이 측정
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 더 이상 스크롤 높이에 변화가 없으면 루프 종료
        if new_height == last_height:
            print("더 이상 로드할 콘텐츠가 없습니다.")
            break
        last_height = new_height

def extract_data(driver):
    """현재 페이지에서 데이터를 추출합니다."""
    data_list = []
    
    try:
        items = driver.find_elements(By.CSS_SELECTOR, DATA_ITEM_SELECTOR)
        print(f"-> 현재 페이지에서 {len(items)}개의 항목 발견.")

        for item in items:
            # TODO: 여기에 실제 웹사이트에 맞게 추출 로직을 작성해야 합니다.
            # 예시: 항목 제목과 URL을 추출하는 코드
            try:
                title = item.find_element(By.CSS_SELECTOR, "h2 a").text
                link = item.find_element(By.CSS_SELECTOR, "h2 a").get_attribute('href')
                
                # 필요에 따라 추가 정보 추출 (예: 가격, 작성일 등)
                
                data_list.append({
                    'Title': title,
                    'Link': link,
                    # 'Price': price
                })
            except NoSuchElementException as e:
                # 데이터 항목 내의 특정 요소가 없을 경우 스킵하거나 기본값 설정
                # print(f"경고: 특정 항목에서 요소 추출 실패 - {e}")
                pass 
                
    except NoSuchElementException:
        print("오류: DATA_ITEM_SELECTOR를 찾을 수 없습니다. 선택자를 확인해주세요.")
    
    return data_list

def main_crawler():
    """메인 크롤링 로직을 실행합니다."""
    driver = setup_driver()
    if not driver:
        return

    all_data = []
    current_page = 1

    try:
        print(f"TARGET_URL로 이동: {TARGET_URL}")
        driver.get(TARGET_URL)
        time.sleep(3) # 페이지 로딩 대기

        while current_page <= MAX_PAGES:
            print(f"\n--- [페이지 {current_page} 크롤링 시작] ---")
            
            # 1. 스크롤 처리 (동적 로딩 데이터 확보)
            scroll_to_bottom(driver)
            
            # 2. 데이터 추출
            page_data = extract_data(driver)
            all_data.extend(page_data)
            
            print(f"-> 페이지 {current_page}에서 {len(page_data)}개 데이터 추출 완료.")
            
            # 3. 페이지네이션 (다음 페이지로 이동)
            if current_page >= MAX_PAGES:
                print(f"최대 페이지 수({MAX_PAGES})에 도달하여 크롤링을 종료합니다.")
                break

            try:
                # 다음 페이지 버튼을 찾기 위해 최대 10초 대기
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, NEXT_PAGE_BUTTON_SELECTOR))
                )
                
                # 버튼이 클릭 가능한 상태인지 확인 후 클릭
                if next_button.is_enabled():
                    print("-> '다음 페이지' 버튼 클릭.")
                    next_button.click()
                    time.sleep(3) # 페이지 로딩 대기
                    current_page += 1
                else:
                    print("-> '다음 페이지' 버튼이 비활성화되었거나 마지막 페이지입니다. 크롤링 종료.")
                    break
                    
            except TimeoutException:
                print("-> '다음 페이지' 버튼을 찾지 못했습니다. 마지막 페이지로 간주하고 크롤링 종료.")
                break
            except NoSuchElementException:
                print("-> '다음 페이지' 버튼 선택자 오류. 크롤링 종료.")
                break

    except Exception as e:
        print(f"크롤링 중 예상치 못한 오류 발생: {e}")
        
    finally:
        driver.quit()
        
    # --- [결과 저장] ---
    if all_data:
        df = pd.DataFrame(all_data)
        output_filename = 'crawling_results.csv'
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n[성공] 총 {len(all_data)}개의 데이터를 {output_filename} 파일에 저장했습니다.")
    else:
        print("\n[완료] 추출된 데이터가 없습니다.")

if __name__ == "__main__":
    main_crawler()


[INFO] page 1: 0 reviews
[INFO] 더 이상 다음 페이지가 없습니다. 종료합니다.
[DONE] 0 rows -> reviews.csv
